# Artificial Intelligence - Assignment 1

## 1. Description

In this assignment, you are going to solve the 8-puzzle using any algorithm. The `EightPuzzle` class is written and provided by TAs, you don't need to implement the puzzle yourself, just import and use.

## 2. How to use `EightPuzzle`

```python
from eight_puzzle import EightPuzzle # import

puzzle = EightPuzzle()

puzzle.init()     # initialize a solvable puzzle state
puzzle.init(seed) # initialize a solvable puzzle state using a seed
print(puzzle)     # show current puzzle state

# move
puzzle.move('up')
puzzle.move('down')
puzzle.move('left')
puzzle.move('right')

if puzzle.state == puzzle.FINAL_STATE:
  print('You have solved the puzzle')

# Useful: get the next state after you move in a direction,
# this won't change the internal state of EightPuzzle.
state_after_move_up = puzzle.get_state_after_move(current_state, 'up')

```



In [66]:
# NOTE: PLEASE KEEP THIS CELL NOT MODIFIED!

# download eight_puzzle.py (YOU SHOULD NOT MODIFY eight_puzzle.py)
!wget https://lab.djosix.com/eight_puzzle.py -qO eight_puzzle.py
!sha1sum eight_puzzle.py

from eight_puzzle import EightPuzzle, test

cea1d62ff26ac1db4b66aea0ac935b8daa0d2422  eight_puzzle.py


## 3. Implement a search algorithm to solve 8-puzzle

In [67]:
import copy
class Node:
    def __init__(self, puzzle, parent_state, prev_direction):
        self.puzzle = puzzle #3x3 2d
        self.current_state = puzzle.state #1d
        self.parent_state = parent_state
        self.goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)
        
        self.f_score = 0
        self.g_score = 0
        self.h_score = 0
        
        self.prev_direction = prev_direction
        self.direction_list = ['up', 'down', 'left', 'right']
        
    def h(self):
        h_score = 0
        for i in range(0, 8):
            cur_row = self.current_state[i]//3
            cur_col = self.current_state[i]%3
            goal_row = self.goal_state[i]//3
            goal_col = self.goal_state[i]%3
            h_score += abs(cur_row - goal_row) + abs(cur_col - goal_col)
            if self.goal_state[i] != self.current_state[i]: #high pentalty for a wrong tile
                h_score += 3
        return h_score
    
    def get_children(self):
        children = []
        for i in range(0, 4):
            if i == 0 and self.prev_direction == 'down':
                continue
            elif i == 1 and self.prev_direction == 'up':
                continue
            elif i == 2 and self.prev_direction == 'right':
                continue
            elif i == 3 and self.prev_direction == 'left':
                continue
            state_after_move = self.puzzle.get_state_after_move(self.current_state, self.direction_list[i])
            if state_after_move != None:
                temp_puzzle = copy.deepcopy(self.puzzle)
                temp_puzzle.move(self.direction_list[i])
                child_node = Node(temp_puzzle, self, self.direction_list[i])
                children.append(child_node)
        return children
                

def solve(p):
    '''Solve the given 8-puzzle and return a list of moves.

    Args:
      puzzle (EightPuzzle): an EightPuzzle object.

    Returns:
      list: a list containing each move direction, for example:
            ['up', 'down', 'left', 'left', ...]
    '''

    # YOUR CODE HERE
    ans = []
    start = Node(p, None, None)
        
    ans = Astar(start)
    return ans

def Astar(start):
    closed_ = []
    open_ = []
    open_.append(start)
    
    solution = []
    
    start.h_score = start.h()
    start.f_score = start.h_score + start.g_score
    
    while(open_!=None):
        min_f_node, min_f_node_idx = min_f(open_)
        # using map() & all() to check if both the lists are exactly equal
        result = all(map(lambda x, y: x == y, min_f_node.current_state, min_f_node.goal_state))
        if result:
            while(min_f_node.prev_direction != None):
                solution.insert(0, min_f_node.prev_direction)
                min_f_node = min_f_node.parent_state
            return solution
        open_.pop(min_f_node_idx)
        closed_.append(min_f_node)
        children = min_f_node.get_children()
        for child in children:
            if child in closed_:
                continue
            tentative_g_score = min_f_node.g_score + 1
            if child not in open_:
                tentative_is_better = True
            elif tentative_g_score < child.g_score:
                tentative_is_better = True
            else:
                tentative_is_better = False
            
            if tentative_is_better == True:
                child.g_score = tentative_g_score
                child.h_score = child.h()
                child.f_score = child.g_score + child.h_score
                open_.append(child)
        
def min_f(searched_list):
    import sys
    min_f_score = sys.maxsize
    idx = 0
    for i in range(len(searched_list)):
        if(searched_list[i].f_score < min_f_score):
            min_f_score = searched_list[i].f_score
            idx = i
    return searched_list[idx], idx

## 4. Test your algorithm

In [68]:
# NOTE: PLEASE KEEP THIS CELL NOT MODIFIED!

results = test(solve, seed=123, n=10)

Running tests with seed: 123
Test | seed: 55260453 | puzzle: (8, 6, 2, 5, 7, 0, 3, 1, 4) | elapsed: 4.4841s | solved
Test | seed: 85291046 | puzzle: (2, 3, 4, 7, 0, 6, 1, 5, 8) | elapsed: 0.1009s | solved
Test | seed: 54801393 | puzzle: (8, 2, 1, 3, 7, 6, 5, 4, 0) | elapsed: 3.1101s | solved
Test | seed: 42905268 | puzzle: (4, 2, 1, 6, 5, 8, 7, 0, 3) | elapsed: 0.3963s | solved
Test | seed: 31981756 | puzzle: (6, 2, 4, 3, 5, 8, 1, 0, 7) | elapsed: 0.9695s | solved
Test | seed: 10224217 | puzzle: (5, 3, 4, 0, 8, 6, 2, 7, 1) | elapsed: 1.0915s | solved
Test | seed: 68555945 | puzzle: (2, 4, 0, 6, 8, 3, 1, 7, 5) | elapsed: 0.0153s | solved
Test | seed: 21759284 | puzzle: (3, 5, 4, 6, 1, 7, 2, 8, 0) | elapsed: 0.6312s | solved
Test | seed: 90208442 | puzzle: (8, 7, 6, 3, 4, 0, 2, 5, 1) | elapsed: 0.8677s | solved
Test | seed: 60679913 | puzzle: (6, 3, 2, 7, 4, 5, 8, 1, 0) | elapsed: 3.4509s | solved
===> Solved: 10/10
===> Average elapsed time: 1.5117s


## 5. Submitting your assignment


1. You have to save 2 files:
  
  - Click "File" > "Download .ipynb" to download `XXXXXXX_1.ipynb`
  - Click "File" > "Download .py" to download `XXXXXXX_1.py`
  
  where `XXXXXXX` is your student ID.  

2. Compress these 2 files into a `.zip` file named `XXXXXXX_1.zip`.

3. Upload the zip file to E3.

> **NOTE: You should click "Runtime" > "Restart and run all" before submitting to E3 to check if your program is able to run without any exception.**